### calculate the leadtime-dependent climatological terciles, deciles and percentiles (0.02, then 0.05 to 0.95 with 0.05 step) from the individual GCMs' hindcast dataset (period 1993 - 2016) for admin areas  

### This notebook is driven via papermill by `ICU_forecast_table/drive_admin_GCMs_evaluation.ipynb`

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

### os and standard libraries 
import os
import sys
from collections import OrderedDict
from itertools import product

### datetimes
from datetime import datetime, timedelta

### scipy
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import cartopy.crs as ccrs
import dask
from dask.diagnostics import ProgressBar
from tqdm import tqdm

### plotting
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns


In [3]:
import pathlib

HOME = pathlib.Path.home()
CWD = pathlib.Path.cwd() 

In [4]:
sys.path.append('../../') 

In [5]:
from ICU_Water_Watch import geo, C3S, domains, plot, utils

### dictionnary holding quantile name and quantile values, they are passed as **lists** to avoid any numerical issues 

In [ ]:
dict_quantiles = OrderedDict()
dict_quantiles['tercile'] = [0.3333, 0.6666]
dict_quantiles['quartile'] = [0.25, 0.5, 0.75]
dict_quantiles['decile'] = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
dict_quantiles['percentile'] = [0.02, 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95] 

### list of GCMs with complete hindcast period 

In [ ]:
GCMs = ['ECMWF', 'UKMO', 'METEO_FRANCE', 'DWD', 'CMCC', 'NCEP', 'JMA', 'ECCC_CanCM4i', 'ECCC_GEM_NEMO']

### PARAMETERS FOR PAPERMILL 

In [ ]:
provider = 'CDS'
varname = 'tprate'
period = 'seasonal'
GCM = 'ECCC_CanCM4i'
quantiles = 'tercile'
country = "Northern Mariana Islands"
admin = "Southern Islands"
method = 'empirical' # whether to calculate the parametrized or empirical quantiles 

In [ ]:
# Parameters
country = "FSM"
admin = "Yap"
GCM = "METEO_FRANCE"
method = "empirical"
period = "seasonal"
quantile = "quartile"


### path definitions follow

### figures 

In [ ]:
fig_path = CWD.parents[1].joinpath("figures/ICU_validation/")

In [ ]:
print(str(fig_path))

### outputs 

In [ ]:
outputs_path = CWD.parents[1].joinpath("outputs/ICU_validation/admin")

In [ ]:
print(str(outputs_path))

#### where to find the GCM hindcast datasets 

In [ ]:
gcm_path = pathlib.Path(f'/media/nicolasf/END19101/ICU/data/{provider}/operational/hindcasts')

In [ ]:
dpath = gcm_path.joinpath(GCM).joinpath(varname.upper())

In [ ]:
print(dpath)

#### where to save the climatologies 

In [ ]:
clim_path = gcm_path.joinpath(f'CLIMATOLOGY/{GCM}')

In [ ]:
print(clim_path)

In [ ]:
if not clim_path.exists(): 
    
    clim_path.mkdir(parents=True)

### get the list of files 

In [ ]:
lfiles = list(dpath.glob(f"ensemble_seas_forecasts_{varname}_from_*.netcdf"))

In [ ]:
lfiles.sort()

In [ ]:
lfiles

In [ ]:
dset = xr.open_mfdataset(lfiles, preprocess=C3S.preprocess_GCM, parallel=True, engine='netcdf4')

In [ ]:
dset = dset.sortby('time')

In [ ]:
dset.info

### print the number of members in the ensemble for each time step 

In [ ]:
# for t in range(len(dset.time)): 
    
#     s = dset.isel(time=t)
    
#     print(f"{pd.to_datetime(dset.isel(time=t)['time'].data):%Y-%m}:", len(s.dropna('member')['member']))

### selects the hindcast period 

In [ ]:
dset = dset.sel(time=slice('1993', '2016'))

In [ ]:
dset

### convert to monthly rainfall accumulations (mm/month)

In [ ]:
dset.tprate.attrs

In [ ]:
dset = C3S.convert_rainfall(dset, varin='tprate', varout='precip', leadvar='step', timevar='time', dropvar=True)

In [ ]:
dset.info

In [ ]:
dset.precip.attrs

### make sure there are no negative values 

In [ ]:
dset = dset.clip(min=0)

### if the period is set to `seasonal`, calculates the seasonal accumulations 

In [ ]:
if period == 'seasonal': 
    
    dset = dset.rolling({'step':3}, min_periods=3).sum('step')
    
    dset = dset.sel({'step':slice(3, None)})

In [ ]:
dset.info

In [ ]:
steps = dset.step.data

In [ ]:
steps

### calculate percentiles over the dimensions member and time, will then be lead time dependent 

In [ ]:
coastlines_dpath = pathlib.Path('/home/nicolasf/operational/ICU/development/hotspots/data/shapefiles/Admin_boundaries/Coastlines')

In [ ]:
shapefile = gpd.read_file(coastlines_dpath.joinpath('ICU_admin_geometries0_360.shp'))

In [ ]:
shapefile

In [ ]:
country_col = 'Country'
admin_col = 'Admin_boun'

In [ ]:
shapefile.loc[:,f"{admin_col}"] = shapefile.loc[:,f"{admin_col}"].str.replace("'","")

### restrict to country 

In [ ]:
sub = shapefile.query(f"Country == '{country}'")

### restrict to administrative area 

In [ ]:
shape = sub.query(f"{admin_col} == '{admin}'")

In [ ]:
shape.plot(figsize=(10,10))

In [ ]:
original_shape = shape.copy()

### for Island groups consituted of very small islands / atolls, we don't filter OUT the geometries, we actually buffer them  so that they can match the resolution of the GCM (once interpolated)

In [ ]:
float(shape.to_crs('EPSG:3857').area / 10**6)

In [ ]:
shape = shape.buffer(0.25)

In [ ]:
# if float(shape.to_crs('EPSG:3857').area / 10**6) < 2000: 
#     shape = shape.buffer(0.25)
# else: 
#     shape = geo.filter_by_area(shape, min_area=500)

In [ ]:
shape.plot()

### we use these filtered geometries to mask the GCM hindcast dataset

#### Note that the GCM dataset is first interpolated to 5X its original resolution 

### do we really need the 15km buffer now that we have the 0.25 degree buffer ? 

In [ ]:
dset, domain = geo.mask_dataset(dset, shape, coastline_buffer=15)

In [ ]:
dset

### number of grid points 

In [ ]:
dset['mask'].attrs['cells']

### plots the shape(s) and the resulting land sea mask  

In [ ]:
if GCM == 'ECMWF': 

    f, axes = plt.subplots(ncols=2, figsize=(10, 10), subplot_kw={'projection':ccrs.PlateCarree(central_longitude=180)})

    ax = axes[0]

    shape.plot(ax=ax, color='0.6', alpha=0.2, transform=ccrs.PlateCarree())

    shape.boundary.plot(ax=ax, color='coral', lw=0.7, transform=ccrs.PlateCarree())

    original_shape.boundary.plot(ax=ax, color='steelblue', lw=0.7, alpha=0.5, transform=ccrs.PlateCarree())

    ax.set_title(f"{country} {admin} coastlines and shape")

#     plot.make_gridlines(ax)

    ax = axes[1]

    dset['mask'].plot(ax=ax, add_colorbar=False, transform=ccrs.PlateCarree(), alpha=0.5)

    original_shape.boundary.plot(ax=ax, color='k', lw=0.7, transform=ccrs.PlateCarree())

    ax.set_title(f"{country} {admin} mask")

#     plot.make_gridlines(ax)

    f.savefig(fig_path.joinpath(f'masks/admin/{utils.sanitize_name(country)}_{utils.sanitize_name(admin)}_shapes_and_mask.png'), dpi=200, bbox_inches='tight', facecolor='w')

### plots the precipitation field for one time step, leadtime and member 

In [ ]:
dset.isel(time=0, step=0, member=0)['precip'].plot()

### calculates the regional average (average over lats and lons)

In [ ]:
dset_sub = dset.mean(['lat','lon'])

### and the calculates the climatological percentiles 

In [ ]:
dset_sub

In [ ]:
dset_sub

### Now calculates the climatological quantiles, either `empirical` or `parametrized` 

In [ ]:
if method == 'parametrized': 
    clim_p = dset_sub.groupby(dset_sub.time.dt.month).apply(C3S.calc_parametrized_quantiles, \
                                                              **{'quantiles':dict_quantiles[quantiles], 'dims':('time','member')})
elif method == 'empirical': 
    clim_p = dset_sub.groupby(dset_sub.time.dt.month).apply(C3S.calc_empirical_quantiles, \
                                                              **{'quantiles':dict_quantiles[quantiles], 'dims':('time','member')})

### plot the climatological quantiles as a function of leadtime ('step' dimension)

In [ ]:
clim_p

In [ ]:
if len(clim_p.month) > 1: 
    clim_p['precip'].plot(x='month', y = 'step', col='quantile', figsize=(10, 4))

### need to save the climatologies 

In [ ]:
outputs_path.joinpath("climatologies")

In [ ]:
print(f"saving {quantiles} climatology for {varname} {period} {GCM} {utils.sanitize_name(country)} {utils.sanitize_name(admin)}")

In [ ]:
clim_p.to_netcdf(outputs_path.joinpath("climatologies").joinpath(f"{method}_{quantiles}_{varname}_{period}_{GCM}_{utils.sanitize_name(country)}_{utils.sanitize_name(admin)}.nc"))

### digitize: each member is given a category depending on the percentile bins defined earlier 

In [ ]:
np.unique(dset_sub.time.dt.month)

In [ ]:
dset_cat = []

for month in np.unique(dset_sub.time.dt.month): 
    
    x = dset_sub.sel(time=(dset_sub.time.dt.month == month))
    
    # drop the missing members 
    
    x = x.dropna('member')
    
    # digitize

    qc = C3S.get_GCM_category_digitize(x, clim_p.sel(month=month), varname='precip', dim='quantile')
    
    dset_cat.append(qc)

### concatenates over the time dimension and reorder 

In [ ]:
dset_cat = xr.concat(dset_cat, dim='time')
dset_cat = dset_cat.sortby('time')

In [ ]:
dset_cat

### Now calculates the probabilities as the proportion of members falling into each category 

In [ ]:
quantiles

In [ ]:
len(dict_quantiles[quantiles]) + 1

In [ ]:
if quantiles == 'tercile': 
    ncategories = 3
if quantiles == 'quartile': 
    ncategories = 4
elif quantiles == 'decile': 
    ncategories = 10 
elif quantiles == 'percentile': 
    ncategories = 21

In [ ]:
quantiles_category_percent = C3S.calculate_quantiles_probabilities(dset_cat, ncategories=ncategories)

In [ ]:
quantiles_category_percent

### because of numerical approximations, sometimes the sum over the quantile dimension is not strictly equal to 100.

In [ ]:
np.alltrue((quantiles_category_percent.sum(quantiles)['precip'] == 100.))

### but it is close enough 

In [ ]:
np.alltrue((quantiles_category_percent.sum(quantiles)['precip'] >= 99.9999))

In [ ]:
np.alltrue((quantiles_category_percent.sum(quantiles)['precip'] <= 100.0001))

In [ ]:
quantiles_category_percent

### saves the percentiles in netcdf 

In [ ]:
if not outputs_path.joinpath(f"hindcast_categories").exists(): 
    outputs_path.joinpath(f"hindcast_categories").mkdir()

In [ ]:
quantiles_category_percent.to_netcdf(outputs_path.joinpath(f"hindcast_categories/{method}_{quantiles}_categories_probabilities_hindcast_{GCM}_{period}_{utils.sanitize_name(country)}_{utils.sanitize_name(admin)}.nc"))

In [ ]:
quantiles_category_percent

In [ ]:
outputs_path

### takes the percentile probabilities then casts into a pandas dataframe 

#### these are the leadtimes, in months, should be [1,2,3,4,5] if monthly, [3,4,5] if seasonal 

In [ ]:
quantiles_category_percent

### transform the xarray dataset into a pandas dataframe with multiindex columns (product of leadtimes and quantiles)

In [ ]:
df_quantile_probabilities = []

for step in steps: 
    
    df = quantiles_category_percent.sel(step=step)['precip'].to_pandas().T
    
    cols = pd.MultiIndex.from_product([[step], df.columns]) 

    df.columns = cols 
    
    df_quantile_probabilities.append(df)


In [ ]:
df_quantile_probabilities = pd.concat(df_quantile_probabilities, axis=1) 

In [ ]:
df_quantile_probabilities.head()

### saves to disk 

In [ ]:
outputs_path

In [ ]:
df_quantile_probabilities.to_csv(outputs_path.joinpath(f"hindcast_categories/{method}_{quantiles}_categories_probabilities_hindcast_{GCM}_{period}_{utils.sanitize_name(country)}_{utils.sanitize_name(admin)}.csv"))

### Now we are going to read the ERA5 reanalysis precipitation data for the validation of the regional aggregates 

In [ ]:
from ICU_Water_Watch import verification

In [ ]:
use_verif = 'era'

In [ ]:
dset_obs, dset_anomalies = verification.get_era5()

In [ ]:
gcm_domain = domains.get_domain(dset)

In [ ]:
gcm_domain

In [ ]:
dset_obs = dset_obs.sel(lon=slice(*gcm_domain[:2]), lat=slice(*gcm_domain[2:]))

In [ ]:
dset_obs

### if seasonal, calculates the 3 months accumulations 

In [ ]:
if period == 'seasonal': 
    
    dset_obs = dset_obs.rolling({"time":3}, min_periods=3, center=False).sum('time')
    
    dset_obs = dset_obs.isel(time=slice(2, None))

In [ ]:
dset_obs['precip'][0,:,:].plot()

In [ ]:
dset_obs = dset_obs.interp_like(dset[['lon','lat']])

In [ ]:
dset_obs

In [ ]:
dset_obs['precip'][0,:,:].plot()

### insert the mask from the GCM dataset

In [ ]:
dset_obs['mask'] = dset['mask']

### same grid and same mask as the GCM 

In [ ]:
dset['mask'].plot()

In [ ]:
dset_obs['mask'].plot()

### apply the mask 

In [ ]:
dset_obs['precip'] = dset_obs['precip'] * dset_obs['mask']

In [ ]:
dset_obs.isel(time=0).squeeze()['precip'].plot()

### insert a dummy "member" dimension with one coordinate

In [ ]:
dset_obs = dset_obs.expand_dims({'member':[1]})

### calculates the regional average 

In [ ]:
dset_obs_sub = dset_obs[['precip']].mean(['lat','lon'])

In [ ]:
dset_obs_sub

In [ ]:
dset_obs_sub = dset_obs_sub.chunk({'member':-1, 'time':-1})

In [ ]:
with ProgressBar(): 
    dset_obs_sub = dset_obs_sub.compute()

### Now calculates the climatological percentiles for the observations, per month 

In [ ]:
clim_obs_p = dset_obs_sub.groupby(dset_obs_sub.time.dt.month).apply(C3S.calc_empirical_quantiles, **{'quantiles':dict_quantiles[quantiles]})

### Now calculates the categories, based on the observations, for each month or season 

In [ ]:
dset_obs_cat = []

for month in np.unique(dset_obs_sub.time.dt.month): 
    
    x = dset_obs_sub.sel(time=(dset_obs_sub.time.dt.month == month))

    # digitize

    qc = C3S.get_GCM_category_digitize(x, clim_obs_p.sel(month=month), varname='precip', dim='quantile')
    
    dset_obs_cat.append(qc)

In [ ]:
dset_obs_cat = xr.concat(dset_obs_cat, dim='time')

In [ ]:
dset_obs_cat = dset_obs_cat.sortby('time')

In [ ]:
dset_obs_cat = dset_obs_cat.squeeze()['precip'].to_pandas()

In [ ]:
dset_obs_cat

In [ ]:
df_quantile_probabilities

In [ ]:
dset_obs_cat = dset_obs_cat.to_frame(name='obs')

In [ ]:
df_quantile_probabilities_obs = dset_obs_cat.join(df_quantile_probabilities, on='time')

In [ ]:
df_quantile_probabilities_obs.to_csv(outputs_path.joinpath(f"{utils.sanitize_name(country)}_{utils.sanitize_name(admin)}_{period}_{GCM}_{quantiles}_probs_and_obs.csv"))

In [ ]:
outputs_path

### do not calculate or save the validations 